### Read Pred & Gold data and evaluate

In [16]:
import os

In [17]:
model_path = '../saved_models/'+'ud_parser_san_2020.02.15-18.59.09_silver_0'

In [18]:
files = os.listdir(model_path)
gold_files =[]
pred_files =[]
for file in files:
    if '.txt'  in file and 'domain' in file and '_res' not in file:
        if 'gold' in file:
            gold_files.append(file)
        elif 'pred' in file:
            pred_files.append(file)
        else:
            continue
        

In [19]:
gold_files

['domain_san_train_model_domain_san_data_domain_san_gold.txt',
 'domain_san_dev_model_domain_san_data_domain_san_gold.txt',
 'domain_san_test_model_domain_san_data_domain_san_gold.txt',
 'domain_san_extra_train_model_domain_san_data_domain_san_gold.txt']

In [20]:
pred_files

['domain_san_extra_train_model_domain_san_data_domain_san_pred.txt',
 'domain_san_test_model_domain_san_data_domain_san_pred.txt',
 'domain_san_dev_model_domain_san_data_domain_san_pred.txt',
 'domain_san_train_model_domain_san_data_domain_san_pred.txt']

In [21]:
f = open(model_path+'/domain_san_train_model_domain_san_data_domain_san_gold.txt','r')
gold = f.readlines()
for i in range(5):
    print(gold[i].split('\t'))

['1', 'na', 'part.', 'Gold_1', '5', 'sambanxah\n']
['2', 'api', 'conj.', 'Gold_1', '5', 'sambanxah\n']
['3', '<_UNK>', 'nom. sg. n.', 'Gold_1', '5', 'karwa\n']
['4', 'taTA', 'adv.', 'Gold_1', '5', 'kriyaviseranam\n']
['5', 'Bavizyati', 'fut. ac. sg. 3', 'Gold_1', '0', 'root\n']


In [22]:
f = open(model_path+'/domain_san_train_model_domain_san_data_domain_san_pred.txt','r')
gold = f.readlines()
for i in range(5):
    print(gold[i].split('\t'))

['1', 'na', 'part.', 'Gold_1', '5', 'sambanxah\n']
['2', 'api', 'conj.', 'Gold_1', '5', 'sambanxah\n']
['3', '<_UNK>', 'nom. sg. n.', 'Gold_1', '5', 'karwa\n']
['4', 'taTA', 'adv.', 'Gold_1', '5', 'kriyaviseranam\n']
['5', 'Bavizyati', 'fut. ac. sg. 3', 'Gold_1', '0', 'root\n']


### Evaluation

In [32]:
def evaluate(gold_file,pred_file):
    f = open(gold_file,'r')
    h = open(pred_file,'r')
    gold = f.readlines()
    pred = h.readlines()
    las=0.0
    uas=0.0
    total=0.0
    for g_line,p_line in zip(gold,pred):
        g_line = g_line.split('\t')
        p_line = p_line.split('\t')
#         print(g_line)
#         print(p_line)
#         print()
        if g_line[0] =='\n':
            continue
        if g_line[1] == p_line[1]:
            if g_line[-2] == p_line[-2]:
                uas+=1
                if g_line[-1] == p_line[-1]:
                    las+=1
            total+=1
    return las/total*100, uas/total*100
print(evaluate(model_path+'/domain_san_dev_model_domain_san_data_domain_san_pred.txt',\
        model_path+'/domain_san_dev_model_domain_san_data_domain_san_gold.txt'))
            
        

(59.81711963423927, 67.23393446786893)


### Build average tag estimate

In [1]:
import fasttext
model = fasttext.load_model("../../fastText/result/cc.sanskrit.300.bin")


In [10]:
import pandas as pd
import os
import numpy as np
dep_tag_dict = dict()
dep_tag_avg  = dict()
for file in os.listdir('../data/sanskrit_treebank/train/'):
    data = pd.read_csv('../data/sanskrit_treebank/train/'+file,sep=',')
    for i in range(len(data)):
        if data.iloc[i,-2] != data.iloc[i,-2] :
            continue
        if data.iloc[i,-2] not in dep_tag_dict.keys():
            dep_tag_dict[data.iloc[i,-2]] = []
            dep_tag_dict[data.iloc[i,-2]].append([data.iloc[i,1],data.iloc[int(data.iloc[i,-1]),1]])
            dep_tag_avg[data.iloc[i,-2]] = np.zeros(600)
            modifier = model.get_word_vector(data.iloc[i,1])
            head     = model.get_word_vector(data.iloc[int(data.iloc[i,-1]),1])
            dep_tag_avg[data.iloc[i,-2]]+= np.concatenate((modifier, head), axis=0)
        else:
            dep_tag_dict[data.iloc[i,-2]].append([data.iloc[i,1],data.iloc[int(data.iloc[i,-1]),1]])
            modifier = model.get_word_vector(data.iloc[i,1])
            head     = model.get_word_vector(data.iloc[int(data.iloc[i,-1]),1])
            dep_tag_avg[data.iloc[i,-2]]+=np.concatenate((modifier, head), axis=0)


In [13]:
for key in dep_tag_dict.keys():
    dep_tag_avg[key] = dep_tag_avg[key]/len(dep_tag_dict[key])

In [23]:
f = open(model_path+'/domain_san_train_model_domain_san_data_domain_san_gold.txt','r')
gold = f.readlines()
for i in range(5):
    print(gold[i].split('\t'))

['1', 'na', 'part.', 'Gold_1', '5', 'sambanxah\n']
['2', 'api', 'conj.', 'Gold_1', '5', 'sambanxah\n']
['3', '<_UNK>', 'nom. sg. n.', 'Gold_1', '5', 'karwa\n']
['4', 'taTA', 'adv.', 'Gold_1', '5', 'kriyaviseranam\n']
['5', 'Bavizyati', 'fut. ac. sg. 3', 'Gold_1', '0', 'root\n']


In [24]:
for line in gold:
    if line == '\n':

['1\tna\tpart.\tGold_1\t5\tsambanxah\n',
 '2\tapi\tconj.\tGold_1\t5\tsambanxah\n',
 '3\t<_UNK>\tnom. sg. n.\tGold_1\t5\tkarwa\n',
 '4\ttaTA\tadv.\tGold_1\t5\tkriyaviseranam\n',
 '5\tBavizyati\tfut. ac. sg. 3\tGold_1\t0\troot\n',
 '\n',
 '1\t<_UNK>\tabl. sg. n.\tGold_10\t2\thewuh\n',
 '2\tBItaH\tnom. sg. m.\tGold_10\t4\tkarwqsamanaxikaranam\n',
 '3\tkfpaRaH\tnom. sg. m.\tGold_10\t4\tkarwa\n',
 '4\tjAyate\tpr. [4] md. sg. 3\tGold_10\t0\troot\n',
 '5\tna\tpart.\tGold_10\t4\tsambanxah\n',
 '6\thi\tpart.\tGold_10\t4\tsambanxah\n',
 '\n',
 '1\taSmaBiH\ti. pl. m.\tGold_1000\t4\tkaranam\n',
 '2\taSvasya\tg. sg. m.\tGold_1000\t3\trartisambanxah\n',
 '3\tgatiH\tnom. sg. f.\tGold_1000\t5\tkarwa\n',
 '4\t<_UNK>\tnom. sg. f.\tGold_1000\t5\tkarwqsamanaxikaranam\n',
 '5\tasti\tpr. [2] ac. sg. 3\tGold_1000\t0\troot\n',
 '\n',
 '1\tyoDaH\tnom. sg. m.\tGold_1001\t4\tkarwa\n',
 '2\tbAREH\ti. pl. m.\tGold_1001\t4\tkaranam\n',
 '3\tarim\tacc. sg. m.\tGold_1001\t4\tkarma\n',
 '4\tjayati\tpr. [1] ac. sg. 3\t